In [1]:
import requests
from bs4 import BeautifulSoup
import os
import time
from tika import parser # tika requiere tener instalado Java 7+
import pandas as pd
dir_discurso = 'discursos'

# Descarga

In [2]:
url_base = 'https://www.banxico.org.mx'
url_discursos = url_base + '/publicaciones-y-prensa/discursos/discursos-junta-gobierno-pala.html'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36'}
resp = requests.get(url_discursos, headers=headers)

In [3]:
soup = BeautifulSoup(resp.content, 'lxml')
documentos = [(tag.select_one('td[class="bmtextview"]').text, tag.select_one('a')['href']) for tag in soup.select('tr')]

In [4]:
for doc in documentos:
    if 'disponible sólo en inglés' in doc[0]:
        continue
    pdfname = f'{dir_discurso}/{doc[1].split("/")[-1]}'
    if not os.path.exists(pdfname):
        time.sleep(1)
        resp = requests.get(url_base+doc[1], headers=headers)
        with open(pdfname, 'wb') as file:
            file.write(resp.content)

# Extrae texto

In [5]:
texto_discursos = []
for doc in documentos:
    if 'disponible sólo en inglés' in doc[0]:
        continue

    pdfname = f'{dir_discurso}/{doc[1].split("/")[-1]}'
    try:
        raw = parser.from_file(pdfname)
        entrada =  {'archivo': pdfname, 'titulo': doc[0],'texto': ' '.join(raw['content'].split()[0:5000])}
        texto_discursos.append(entrada)
    except:
        print('problema con', pdfname)

2019-11-22 11:58:26,604 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


In [8]:
df = pd.DataFrame(data=texto_discursos)
df.to_csv('../datos/discursos.csv', index=False)
df.head()

,archivo,texto,titulo
0,discursos/{0CF3D30D-2EE7-81DE-5A25-79D9D3391F3...,"Palabras del Lic. Alejandro Díaz de León, Gobe...","""La importancia de la autonomía de la banca ce..."
1,discursos/{EA967153-2AC4-97F2-C6BB-28634631C67...,"Palabras del Lic. Alejandro Díaz de León, Gobe...","""Inauguración de la XI Conferencia sobre Educa..."
2,discursos/{B11F2552-BDB5-BC21-3DDF-FB62AE2C341...,"Palabras del Lic. Alejandro Díaz de León, Gobe...","""Ceremonia de inauguración de la exposición Cr..."
3,discursos/{813918D3-3710-EE26-B5EB-E800B21427F...,"Palabras del Lic. Alejandro Díaz de León, Gobe...","""Inauguración del seminario sobre Estabilidad ..."
4,discursos/{822B0923-7213-CA97-8B63-DA6F24FA08C...,"Palabras del Lic. Alejandro Díaz de León, Gobe...","""Inauguración de la Cuarta Convención Nacional..."
